In [1]:
!pip install numpy pandas matplotlib seaborn scikit-learn

  Using cached matplotlib-3.10.8-cp310-cp310-win_amd64.whl (8.1 MB)
  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
  Using cached pillow-12.0.0-cp310-cp310-win_amd64.whl (7.0 MB)
  Using cached kiwisolver-1.4.9-cp310-cp310-win_amd64.whl (73 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached contourpy-1.3.2-cp310-cp310-win_amd64.whl (221 kB)
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     --- ------------------------------------ 0.2/1.6 MB 9.0 MB/s eta 0:00:01
     --------- ------------------------------ 0.4/1.6 MB 3.7 MB/s eta 0:00:01
     ---------------- ----------------------- 0.6/1.6 MB 5.0 MB/s eta 0:00:01
     --------------------- ------------------ 0.8/1.6 MB 4.9 MB/s eta 0:00:01
     -------------------------- ------------- 1.0/1.6 MB 4.4 MB/s eta 0:00:01
     ----------------------------- ---------- 1.1/1.6 MB 4.2 MB/s eta 0:00:01
     ------------------------------- -------- 1.2/1.6 MB 3.9 MB/s eta 0:00:01
     ---


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# ----------------------------
# 1. LOAD DATASET
# ----------------------------
FILE_PATH = "fundamental.csv"   # change if filename differs

df = pd.read_csv(FILE_PATH)

print("\n========== BASIC INFO ==========")
print(df.info())

print("\n========== SHAPE ==========")
print(df.shape)

print("\n========== FIRST 5 ROWS ==========")
print(df.head())

,symbol,revenuePerShare,trailingPE,earningsQuarterlyGrowth,previousClose,open,dayLow,dayHigh,volume,trailingEps,pegRatio,ebitda,totalDebt,totalRevenue,debtToEquity,revenuePerShare.1,earningsGrowth,revenueGrowth
0,RELIANCE.BO,1296.823,28.012129,0.093,2901.30,2897.05,2895.35,2920.0,562484.0,103.88,NaN,1.503867e+12,3.166970e+12,8.773650e+12,36.100,1296.823,0.093,0.036
1,RELIANCE.NS,1296.823,28.010878,0.093,2901.95,2899.95,2894.70,2920.0,9763420.0,103.89,NaN,1.503867e+12,3.166970e+12,8.773650e+12,36.100,1296.823,0.093,0.036
2,HDFCBANK.BO,293.740,16.260424,0.359,1445.10,1437.30,1437.30,1450.7,783356.0,88.74,NaN,NaN,7.996598e+12,1.933224e+12,NaN,293.740,-0.001,1.211
3,HINDUNILVR.NS,263.412,51.285230,0.014,2242.35,2239.05,2232.05,2266.0,3507581.0,44.00,NaN,1.421525e+11,1.272000e+10,6.190100e+11,2.515,263.412,0.014,-0.002
4,ICICIBANK.NS,197.399,18.315136,0.257,1081.80,1081.15,1078.70,1093.7,17212189.0,59.53,1.03,NaN,2.009669e+12,1.380849e+12,NaN,197.399,0.253,0.204


In [7]:
# ----------------------------
# 2. COLUMN NAMES CLEANING
# ----------------------------
df.columns = (
    df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace("%", "Percent")
)

print("\n========== CLEANED COLUMN NAMES ==========")
print(df.columns.tolist())


========== CLEANED COLUMN NAMES ==========
['symbol', 'revenuePerShare', 'trailingPE', 'earningsQuarterlyGrowth', 'previousClose', 'open', 'dayLow', 'dayHigh', 'volume', 'trailingEps', 'pegRatio', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare.1', 'earningsGrowth', 'revenueGrowth']


In [8]:
# ----------------------------
# 3. CHECK MISSING VALUES
# ----------------------------
print("\n========== MISSING VALUES ==========")
missing = df.isnull().sum()
print(missing[missing > 0])


========== MISSING VALUES ==========
revenuePerShare            2675
trailingPE                 2528
earningsQuarterlyGrowth    3584
previousClose               580
open                        578
dayLow                      578
dayHigh                     578
volume                      578
trailingEps                2668
pegRatio                   5557
ebitda                     2898
totalDebt                  2839
totalRevenue               2676
debtToEquity               3170
revenuePerShare.1          2675
earningsGrowth             3622
revenueGrowth              2739
dtype: int64


In [9]:
# ----------------------------
# 4. SEPARATE NUMERIC & CATEGORICAL
# ----------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns
categorical_cols = df.select_dtypes(exclude=[np.number]).columns

print("\nNumeric columns:")
print(list(numeric_cols))

print("\nCategorical columns:")
print(list(categorical_cols))


Numeric columns:
['revenuePerShare', 'trailingPE', 'earningsQuarterlyGrowth', 'previousClose', 'open', 'dayLow', 'dayHigh', 'volume', 'trailingEps', 'pegRatio', 'ebitda', 'totalDebt', 'totalRevenue', 'debtToEquity', 'revenuePerShare.1', 'earningsGrowth', 'revenueGrowth']

Categorical columns:
['symbol']


In [10]:
# ----------------------------
# 5. HANDLE MISSING VALUES
# ----------------------------

# Numeric → median (robust)
for col in numeric_cols:
    if df[col].isnull().sum() > 0:
        median_value = df[col].median()
        df[col].fillna(median_value, inplace=True)

# Categorical → mode
for col in categorical_cols:
    if df[col].isnull().sum() > 0:
        mode_value = df[col].mode()[0]
        df[col].fillna(mode_value, inplace=True)

print("\n========== MISSING VALUES AFTER CLEANING ==========")
print(df.isnull().sum().sum(), "total null values")


========== MISSING VALUES AFTER CLEANING ==========
0 total null values


C:\Users\Jayalakshmi\AppData\Local\Temp\ipykernel_10116\648638251.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median_value, inplace=True)
C:\Users\Jayalakshmi\AppData\Local\Temp\ipykernel_10116\648638251.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For

In [12]:
# ----------------------------
# 6. SYMBOL COLUMN CHECK (VERY IMPORTANT)
# ----------------------------
if "symbol" in df.columns:
    df["symbol"] = df["symbol"].str.strip().str.upper()
else:
    raise Exception("❌ SYMBOL column missing. This dataset is not suitable for watchlist.")

print("\nUnique companies count:", df["symbol"].nunique())


Unique companies count: 5829


In [13]:
# ----------------------------
# 7. REMOVE DUPLICATES (IF ANY)
# ----------------------------
before = df.shape[0]
df.drop_duplicates(inplace=True)
after = df.shape[0]

print(f"\nDuplicates removed: {before - after}")


Duplicates removed: 81


In [14]:
# ----------------------------
# 8. DESCRIPTIVE STATISTICS (EDA CORE)
# ----------------------------
print("\n========== DESCRIPTIVE STATS ==========")
print(df.describe())


========== DESCRIPTIVE STATS ==========
       revenuePerShare   trailingPE  earningsQuarterlyGrowth  previousClose  \
count      5829.000000  5829.000000              5829.000000    5829.000000   
mean        264.382839          inf                 0.620612     618.090009   
std        1181.765731          NaN                 7.249778    3064.934828   
min         -28.928000     0.025160                -0.998000       0.030000   
25%         111.958000    26.179459                 0.187000      35.100000   
50%         135.804000    29.994380                 0.187000     126.525000   
75%         162.387000    34.973700                 0.187000     427.400000   
max       58164.040000          inf               311.196000  131639.050000   

                open         dayLow        dayHigh        volume  trailingEps  \
count    5829.000000    5829.000000    5829.000000  5.829000e+03  5829.000000   
mean      619.056457     611.511247     632.486455  7.092336e+05    14.007071   
std 

c:\Users\Jayalakshmi\Desktop\infosys\.venv\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [15]:
# ----------------------------
# 9. SAVE CLEANED DATASET
# ----------------------------
OUTPUT_FILE = "fundamental_cleaned.csv"
df.to_csv(OUTPUT_FILE, index=False)

print(f"\nCLEANED DATASET SAVED AS: {OUTPUT_FILE}")


CLEANED DATASET SAVED AS: fundamental_cleaned.csv
